# Creando un poema con Keras


Natural Language Processing - Tokenization (NLP Zero to Hero - Part 1) : https://www.youtube.com/watch?v=fNxaJsNG3-s&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S

El objetivo es crear un programa con tensorflow y keras, que genere textos coherentes de poesía mediante una red neuronal que prediga palabras en función de otras.

## 0 - Import de librerías generales y carga de textos

In [32]:
import tensorflow 
from tensorflow import keras
import numpy as np
import requests
import random
import os

In [33]:
base_url  = 'https://raw.githubusercontent.com/MrCabss69/KerasTextClassification/main/resources/'
train_url = base_url + 'train.txt'
test_url  = base_url + 'test.txt'
textos    = []
for url in [train_url,test_url]:
  textos += requests.get(url).content.decode("utf-8").split('\n')
  textos.remove('')

In [34]:
N = len(textos)
print('Muestras totales: ',len(textos))
print(textos[:10])
random.shuffle(textos)

Muestras totales:  18000
['i didnt feel humiliated;sadness', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake;sadness', 'im grabbing a minute to post i feel greedy wrong;anger', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property;love', 'i am feeling grouchy;anger', 'ive been feeling a little burdened lately wasnt sure why that was;sadness', 'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny;surprise', 'i feel as confused about life as a teenager or as jaded as a year old man;fear', 'i have been with petronas for years i feel that petronas has performed well and made a huge profit;joy', 'i feel romantic too;love']


## 1 - Tokenización y preprocesamiento de texto

 Los inputs para el entrenamiento del modelo serán cadenas de texto (frases), y serán de tamaño fijo, teniendo el vocabulario de entrenamiento un tamaño limitado. Debemos transformar los textos en vectores numéricos codificando las palabras individuales del texto.

 Keras implementa la clase Tokenizer de preprocessin/text con varios métodos útiles predefinidos.
 


In [35]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, regularizers

In [36]:
# creamos el tokenizador sin límite de palabras
tkn = Tokenizer()

# ajustamos a los textos
tkn.fit_on_texts(textos)

# ya tenemos nuestro vocabulario indexado
w_idx = tkn.word_index

print(len(w_idx))
print(w_idx)

16184
{'i': 1, 'feel': 2, 'and': 3, 'to': 4, 'the': 5, 'a': 6, 'joy': 7, 'feeling': 8, 'that': 9, 'of': 10, 'sadness': 11, 'my': 12, 'in': 13, 'it': 14, 'like': 15, 'so': 16, 'im': 17, 'for': 18, 'me': 19, 'have': 20, 'but': 21, 'was': 22, 'anger': 23, 'is': 24, 'this': 25, 'am': 26, 'with': 27, 'fear': 28, 'not': 29, 'about': 30, 'love': 31, 'be': 32, 'as': 33, 'on': 34, 'you': 35, 'just': 36, 'at': 37, 'when': 38, 'or': 39, 'all': 40, 'because': 41, 'more': 42, 'do': 43, 'can': 44, 'really': 45, 'up': 46, 't': 47, 'by': 48, 'are': 49, 'very': 50, 'been': 51, 'know': 52, 'if': 53, 'out': 54, 'myself': 55, 'time': 56, 'what': 57, 'how': 58, 'get': 59, 'little': 60, 'had': 61, 'now': 62, 'will': 63, 'from': 64, 'they': 65, 'being': 66, 'people': 67, 'he': 68, 'them': 69, 'would': 70, 'want': 71, 'her': 72, 'some': 73, 'one': 74, 'still': 75, 'think': 76, 'ive': 77, 'who': 78, 'surprise': 79, 'even': 80, 'him': 81, 'an': 82, 'life': 83, 'its': 84, 'there': 85, 'make': 86, 'bit': 87, 'we'

In [37]:
seq = tkn.texts_to_sequences(textos)
print(seq)

[[1, 93, 20, 4, 724, 14, 13, 7940, 111, 125, 2453, 23], [77, 51, 8, 6, 87, 553, 41, 596, 16, 60, 56, 174, 18, 227, 622, 28], [1, 104, 47, 71, 81, 4, 2, 3279, 39, 800, 11], [1, 26, 1345, 13, 12, 7941, 7942, 2454, 5624, 1486, 2455, 1594, 46, 537, 215, 3764, 1815, 7943, 64, 2456, 3, 26, 194, 4, 2, 1595, 3, 1733, 7], [17, 302, 37, 1596, 41, 2284, 1281, 24, 5625, 760, 210, 67, 2, 684, 11], [1, 2, 15, 25, 1024, 63, 191, 145, 19, 1025, 6, 608, 781, 9, 24, 2285, 249, 33, 141, 33, 7944, 4, 12, 962, 303, 7], [1, 20, 4, 768, 27, 5, 294, 9, 1281, 898, 275, 4, 2, 15, 65, 355, 13, 669, 31, 18, 6, 609, 10, 157, 3, 128, 88, 44, 40, 1000, 57, 761, 49, 11], [1, 2, 17, 29, 429, 48, 9, 23], [17, 8, 211, 4472, 3, 750, 11], [1, 20, 51, 8, 750, 3, 5626, 11], [1, 116, 16, 160, 67, 78, 449, 127, 37, 5, 1346, 10, 6, 2037, 41, 14, 94, 36, 6, 328, 3, 29, 50, 290, 4, 113, 1393, 66, 3, 9, 94, 390, 21, 1, 20, 4, 43, 89, 9, 1, 2, 24, 714, 4, 145, 19, 526, 34, 57, 1, 4473, 33, 6, 109, 1239, 7], [1, 22, 8, 769, 7], [1,

Para comparar frases de diferente longitud, o tener muestras con el mismo número de variables/columnas hace falta rellenar a 0's las posiciones de los strings más cortos. Se coge la longitud más larga como tamaño máximo (número inicial de columnas)

In [38]:
padded = np.array(pad_sequences(seq, padding='pre',maxlen=max([len(s) for s in seq])))
s = padded.shape

In [39]:
print(padded.shape)
print('Hay actualmente ',s[0],' frases de muestra')
print('El vocabulario actualmente tiene una longitud de ',s[1])

(18000, 67)
Hay actualmente  18000  frases de muestra
El vocabulario actualmente tiene una longitud de  67


Las opciones de la función de padding son realizar el rellanado a 0's en las columnas de mayor índice (padding='post'), y dar una anchura máxima de palabra distinta de la encontrada en los textos (maxlen=5)

## 2 - Separación de datos para entrenamiento

In [40]:
idx_w = {v:k for k,v in w_idx.items()}

In [41]:
# train_s, test_s, val_s
T_SIZE = int(0.9*N)
train_x, labels = padded[:T_SIZE,:-1],padded[:T_SIZE,-1]

In [42]:
print('Primera muestra:',train_x[0])
print('En texto: ',[ idx_w[w] for w in train_x[0] if w != 0])
print('Categoría',idx_w[labels[0]])

Primera muestra: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    1
   93   20    4  724   14   13 7940  111  125 2453]
En texto:  ['i', 'dont', 'have', 'to', 'buy', 'it', 'in', 'tubs', 'which', 'feels', 'vile']
Categoría anger


In [43]:
#train_x = train_x / np.linalg.norm(train_x)
train_y = tensorflow.keras.utils.to_categorical(labels)
print(train_x.shape)
print(train_y.shape)

(16200, 66)
(16200, 80)


## 3 -Creando el modelo

Código del modelo sacado de la clase de TF

In [44]:
n_filters= 10
k_size = 100
p_size = 10

In [45]:
model = Sequential([
    Embedding(input_dim=train_x.shape[1],output_dim=200),
    Bidirectional(LSTM(units=150,kernel_regularizer=regularizers.L2(l=0.01))),
    Dropout(rate=0.1),
    Dense(units=200,activation='relu'),
    Dropout(rate=0.2),
    Dense(units=train_y.shape[1],activation='sigmoid')
])
model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
model.build()
history = model.fit(train_x, train_y, epochs=1, verbose=1)


507/507 [==============================] - 9s 12ms/step - loss: 1.9851 - accuracy: 0.3145


In [46]:
#tensorflow.keras.utils.plot_model(model)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 200)         13200     
                                                                 
 bidirectional_2 (Bidirectio  (None, 300)              421200    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 200)               60200     
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_5 (Dense)             (None, 80)                16080     
                                                      

## 4 - Evaluación y testeo del modelo



In [57]:
test_x, test_y = padded[T_SIZE:T_SIZE+500,:-1],padded[T_SIZE:T_SIZE+500,-1]
test_y = tensorflow.keras.utils.to_categorical(test_y)
print(test_x.shape)
print(test_y.shape)


ValueError: ignored

In [48]:
model.evaluate(x=test_x,y=test_y,verbose=1)

16/16 [==============================] - 1s 6ms/step - loss: 1.5511 - accuracy: 0.3260


[1.5511212348937988, 0.32600000500679016]

In [49]:
sample_text = ' It is a cloudy day'
tkn2 = Tokenizer()
tkn2.fit_on_texts([sample_text])
dw = tkn2.word_index

In [50]:
input = np.array([w_idx[k] for k in dw.keys()])
print(input)

[  14   24    6 7042  101]


In [51]:
padded = pad_sequences([input],maxlen=len(w_idx))
print(padded.shape)
print(padded)
padded = padded / np.linalg.norm(padded)


(1, 16184)
[[   0    0    0 ...    6 7042  101]]


In [52]:
p = model.predict(padded)
print(p.shape)

1/1 [==============================] - 2s 2s/step
(1, 80)


In [53]:
l = np.where(p[0] == max(p[0]))[0]

In [54]:
print(l)
print(idx_w[l[0]])

[7]
joy


In [55]:
idx = np.unravel_index(np.argmax(p), p.shape)

In [56]:
print(idx)
print()

(0, 7)

